In [ ]:
import pandas as pd

# 1) Wczytanie debug-pliku
df = pd.read_csv(
    'exerfly-1749483170071-debug.csv',
    skipinitialspace=True
)

# 2) Upewniamy się, że kolumny są numeryczne
df['Angle']    = pd.to_numeric(df['Angle'], errors='raise')
df['Time (µs)'] = pd.to_numeric(df['Time (µs)'], errors='raise')

# 3) Łączymy dwa pola w jeden łańcuch "angle,time"
#    Dzięki sep=';' poniższy plik nie będzie otoczony cudzysłowami
out = pd.DataFrame()
out['time,angle'] = (
    df['Time (µs)'].astype(str)
    + ',' +
    df['Angle'].astype(str)
)

# 4) Zapis do CSV obok notebooka – używamy średnika jako delimiter,
#    żeby wartości wewnątrz pola (oddzielone przecinkiem) nie były cytowane.
out.to_csv('angle_time_combined.csv', index=False, sep=';')

print("Gotowe – wygenerowano plik 'angle_time_combined.csv'")


Gotowe – wygenerowano plik 'angle_time_combined.csv'


In [36]:
import pandas as pd

# --- 1) Wczytanie pierwszego pliku z metadanymi ---
df1 = pd.read_csv(
    'exerfly-1749460260552.csv',
    skiprows=16,                          # pomijamy pierwsze 16 linii
    usecols=['Time (µs)', 'Angle (°)']    # tylko te dwie kolumny
).rename(
    columns={'Time (µs)': 'time', 'Angle (°)': 'shaftAngle'}
)

# --- 2) Wczytanie drugiego pliku ("debug") ---
df2 = pd.read_csv(
    '0.1-exerfly-1749460474580-debug.csv',
    usecols=['time', 'shaftAngle']        # plik już ma dwie kolumny
)
# Upewniamy się, że są typami numerycznymi
df2['time'] = pd.to_numeric(df2['time'], errors='raise')
df2['shaftAngle'] = pd.to_numeric(df2['shaftAngle'], errors='raise')

# --- 3) Porównanie liczby rekordów ---
n1 = len(df1)
n2 = len(df2)
diff = abs(n1 - n2)

# --- 4) Identyfikacja unikalnych rekordów ---
only_in_df1 = (
    df1.merge(df2, on=['time','shaftAngle'], how='left', indicator=True)
       .query("_merge == 'left_only'")
       .drop(columns=['_merge'])
)
only_in_df2 = (
    df2.merge(df1, on=['time','shaftAngle'], how='left', indicator=True)
       .query("_merge == 'left_only'")
       .drop(columns=['_merge'])
)

# --- 5) Zapis wyników do plików ---
only_in_df1.to_csv('only_in_df1.csv', index=False)
only_in_df2.to_csv('only_in_df2.csv', index=False)

# --- 6) Zapis podsumowania porównania ---
with open('comparison_summary.txt', 'w', encoding='utf-8') as f:
    f.write(f"Liczba rekordów w df1: {n1}\n")
    f.write(f"Liczba rekordów w df2: {n2}\n")
    f.write(f"Różnica: {diff} rekordów\n")
    f.write(f"Rekordy unikalne w df1: {len(only_in_df1)}\n")
    f.write(f"Rekordy unikalne w df2: {len(only_in_df2)}\n")

print("Zapisano pliki:")
print(" - only_in_df1.csv          (rekordy tylko w df1)")
print(" - only_in_df2.csv          (rekordy tylko w df2)")
print(" - comparison_summary.txt   (podsumowanie porównania)")


Zapisano pliki:
 - only_in_df1.csv          (rekordy tylko w df1)
 - only_in_df2.csv          (rekordy tylko w df2)
 - comparison_summary.txt   (podsumowanie porównania)
